<img src="https://raw.githubusercontent.com/tamedia-ddj/brennpunkt_bauernhof_public/master/files/Bauernhof_Logo.jpg">

# Brennpunkt Bauernhof
## Analyse des Datenbestandes zu den Direktzahlungen 2014 bis 2017

Ein Rechercheteam von Tamedia veröffentlich diese Woche eine Serie von Artikeln zur Landwirtschaft. Es wurden dazu Daten über Direktzahlung sowie über 600 Strafurteile gegen Landwirte ausgewertet.

In den folgenden Notebooks sind die relevanten Ausschnitte der Analyse dokumentiert. Der beiliegende Datensatz wurde zur Wahrung des Datenschutzes gekürzt und verändert (Gemeinde, Kantone und Zahlenwerte wurden willkürlich vertauscht).




1. [Datenvorbereitung](https://github.com/tamedia-ddj/brennpunkt_bauernhof_public/blob/master/0_Data_Preparation.ipynb)
2. Kürzungen
    * Methodik: [Github](https://github.com/tamedia-ddj/brennpunkt_bauernhof_public/blob/master/1_Kuerzungen.ipynb)
    * Artikel:
        * [Skandalöse Zustände auf Schweizer Bauernhöfen](https://www.tagesanzeiger.ch/schweiz/brennpunkt-bauernhof/skandaloese-zustaende-auf-schweizer-bauernhoefen/story/17687029)
        * [Sogar Tierquäler erhalten Subventionen](https://www.tagesanzeiger.ch/schweiz/brennpunkt-bauernhof/bauern-erhalten-subventionen-trotz-leidender-tiere/story/19918846)
3. Kontrollen
	* Artikel:
		* [Zu wenig Kontrollen: Tierquäler bleiben unentdeckt](https://www.tagesanzeiger.ch/schweiz/brennpunkt-bauernhof/zu-wenig-geld-fuer-tierkontrollen/story/12827795)
        * [Der Bauern-Sherlock](https://www.tagesanzeiger.ch/schweiz/brennpunkt-bauernhof/wie-ein-ermittler-mit-videobeweisen-landwirte-ueberfuehrt/story/16595684)
4. **Direktzahlungsbeträge**
    * Artikel: [«Das war für uns wie ein Lottosechser»](https://www.tagesanzeiger.ch/schweiz/brennpunkt-bauernhof/welche-bauern-am-meisten-erhalten/story/25753875)
5. Interview
	* Artikel: [«Beim Tierschutz gibt es keine Schmerzgrenze»](https://www.tagesanzeiger.ch/schweiz/brennpunkt-bauernhof/beim-tierschutz-gibt-es-keine-schmerzgrenze/story/20647149)
6. Interaktiv
    * Artikel: [Ein Leben lang trächtig und am Ende eine Wurst](https://interaktiv.tagesanzeiger.ch/2019/leben-einer-kuh/)
7. Tiertransporte
    * Artikel: [Die letzte Reise von Schwein Fanny](https://www.tagesanzeiger.ch/schweiz/brennpunkt-bauernhof/die-letzte-reise-von-schwein-fanny/story/14773958)



Quelle des Datensatzes: BLW

---

### Notebook 3 - DZ-Beträge
#### Berechnungen zu Artikeln

---

#### Load Packages

In [3]:
# Import necessary packages
import pandas as pd
import numpy as np

#### Import data

In [4]:
# read pickle-file
data = pd.read_pickle("files/data_mod.pkl")
# import fixed list of municipalities and their corresponding cantons
clean_gde = pd.read_pickle("files/gemeinden_bereinigt.pkl")

---

#### initial jobs

##### create function

In [27]:
### function to clean municipality name / remove canton in parentheses
import re
def remove_kte(gde):
    return re.sub(r" ?\([^)]+\)", "", gde)

In [28]:
### function to get total number of farms by GDE_GEMEINDE_NR
def get_total_betriebe(gde_nr):
    return len(data[data.GDE_GEMEINDE_NR == gde_nr].BETRIEBSNUMMER_ZUFALL.unique())

In [29]:
### function to remove decimals
def clean_decimal(amount):
    return ("%.0f" % amount)

In [30]:
### function to reformat numbers into string and add a thousands separator
def clean_nr(amount):
    amount = (str("%.0f" % amount))
    return ("{}'{}".format(amount[:3], amount[3:]))

##### import list of farms with more than 250k subisdies which are not cases of group farming (Betriebsgemeinschaften)
will be used later on to filter out all entries which consist of more then 1 farm

In [75]:
### get BETRIEBSNUMMER_ZUFALL of all single farms with more than 250k subsidies
betriebe250k_official = pd.read_excel("data_input/DZ_250000_ohne_BG_mod.xlsx", index_col=0)
betriebe250k_official = betriebe250k_official[["BETRIEBSNUMMER_ZUFALL"]]


In [32]:
### get a list of all group farming entries
betriebe250k_exclude = data[(data["JAHR"] == 2017) & (data.DZ_BEITRAG_BUND > 250000) & (~data.BETRIEBSNUMMER_ZUFALL.isin(betriebe250k_official.BETRIEBSNUMMER_ZUFALL))][["BETRIEBSNUMMER_ZUFALL"]]


---

#### calculate number of farms in 4 income groups (subsidies in 2017 less than 50k, 150k, 250k or more than 250k)

In [33]:
### income group 1 (less than 50k)
### calculate number of farms and sum and mean of corresping subisdies on municipal level
gde_dz_to50k = data[(data["JAHR"] == 2017) & (data.DZ_BEITRAG_BUND <= 50000) & (data.DZ_BEITRAG_BUND > 0)][["GDE_GEMEINDE_NR", "DZ_BEITRAG_BUND"]].groupby(["GDE_GEMEINDE_NR"]).agg(['count', 'sum', 'mean'])
gde_dz_to50k.columns = ['_'.join(t) for t in gde_dz_to50k.columns]
gde_dz_to50k = pd.DataFrame(gde_dz_to50k.to_records())


In [34]:
### income group 2 (less than 150k)
### calculate number of farms and sum and mean of corresping subisdies on municipal level
gde_dz_50to150k = data[(data["JAHR"] == 2017) & (data.DZ_BEITRAG_BUND > 50000) & (data.DZ_BEITRAG_BUND <= 150000)][["GDE_GEMEINDE_NR", "DZ_BEITRAG_BUND"]].groupby(["GDE_GEMEINDE_NR"]).agg(['count', 'sum', 'mean'])
gde_dz_50to150k.columns = ['_'.join(t) for t in gde_dz_50to150k.columns]
gde_dz_50to150k = pd.DataFrame(gde_dz_50to150k.to_records())


In [35]:
### income group 3 (less than 250k)
### calculate number of farms and sum and mean of corresping subisdies on municipal level
gde_dz_150to250k = data[(data["JAHR"] == 2017) & (data.DZ_BEITRAG_BUND > 150000) & (data.DZ_BEITRAG_BUND <= 250000)][["GDE_GEMEINDE_NR", "DZ_BEITRAG_BUND"]].groupby(["GDE_GEMEINDE_NR"]).agg(['count', 'sum', 'mean'])
gde_dz_150to250k.columns = ['_'.join(t) for t in gde_dz_150to250k.columns]
gde_dz_150to250k = pd.DataFrame(gde_dz_150to250k.to_records())


In [36]:
### income group 4 (more than 250k)
### apply filter to remove group farming
### calculate number of farms and sum and mean of corresping subisdies on municipal level
gde_dz_from250k = data[(data["JAHR"] == 2017) & data.BETRIEBSNUMMER_ZUFALL.isin(betriebe250k_official.BETRIEBSNUMMER_ZUFALL)][["GDE_GEMEINDE_NR", "DZ_BEITRAG_BUND"]].groupby(["GDE_GEMEINDE_NR"]).agg(['count', 'sum', 'mean'])
gde_dz_from250k.columns = ['_'.join(t) for t in gde_dz_from250k.columns]
gde_dz_from250k = pd.DataFrame(gde_dz_from250k.to_records())


In [37]:
### calculate number of all farms and sum and mean of corresping subisdies on municipal level
gde_dz_all = data[(data["JAHR"] == 2017) & (~data.BETRIEBSNUMMER_ZUFALL.isin(betriebe250k_exclude.BETRIEBSNUMMER_ZUFALL)) & (data.DZ_BEITRAG_BUND > 0)][["GDE_GEMEINDE_NR", "DZ_BEITRAG_BUND"]].groupby(["GDE_GEMEINDE_NR"]).agg(['count', 'sum', 'mean'])
gde_dz_all.columns = ['_'.join(t) for t in gde_dz_all.columns]
gde_dz_all = pd.DataFrame(gde_dz_all.to_records())


##### merge data into new dataframe gd_dz_groups on municipal level

In [38]:
### collect matching name and canton and merge with income group 1
gd_dz_groups = pd.merge(clean_gde, gde_dz_to50k[["GDE_GEMEINDE_NR", "DZ_BEITRAG_BUND_count", "DZ_BEITRAG_BUND_sum", "DZ_BEITRAG_BUND_mean"]], on="GDE_GEMEINDE_NR", how="left", suffixes=('', 'to50k'))


In [39]:
### merge gd_dz_groups with income group 2
gd_dz_groups = pd.merge(gd_dz_groups, gde_dz_50to150k[["GDE_GEMEINDE_NR", "DZ_BEITRAG_BUND_count", "DZ_BEITRAG_BUND_sum", "DZ_BEITRAG_BUND_mean"]], on="GDE_GEMEINDE_NR", how="left", suffixes=('', '50to150k'))


In [40]:
### merge gd_dz_groups with income group 3
gd_dz_groups = pd.merge(gd_dz_groups, gde_dz_150to250k[["GDE_GEMEINDE_NR", "DZ_BEITRAG_BUND_count", "DZ_BEITRAG_BUND_sum", "DZ_BEITRAG_BUND_mean"]], on="GDE_GEMEINDE_NR", how="left", suffixes=('', '150to250k'))


In [41]:
### merge gd_dz_groups with income group 4
gd_dz_groups = pd.merge(gd_dz_groups, gde_dz_from250k[["GDE_GEMEINDE_NR", "DZ_BEITRAG_BUND_count", "DZ_BEITRAG_BUND_sum", "DZ_BEITRAG_BUND_mean"]], on="GDE_GEMEINDE_NR", how="left", suffixes=('', 'from250k'))


In [42]:
### merge gd_dz_groups with summary statistics on municipal level
gd_dz_groups = pd.merge(gd_dz_groups, gde_dz_all[["GDE_GEMEINDE_NR", "DZ_BEITRAG_BUND_count", "DZ_BEITRAG_BUND_sum", "DZ_BEITRAG_BUND_mean"]], on="GDE_GEMEINDE_NR", how="left", suffixes=('', 'all'))


In [43]:
### rename columns
gd_dz_groups.columns = ["GDE_GEMEINDE_NR", "GDE_NAME", "KANTON", "count_to50k", "sum_to50k", "mean_to50k", "count_50to150k", "sum_50to150k", "mean_50to150k", "count_150to250k", "sum_150to250k", "mean_150to250k", "count_from250k", "sum_from250k", "mean_from250k", "count", "sum", "mean"]


In [44]:
### replace NAs with value 0
gd_dz_groups = gd_dz_groups.fillna(0)

##### number of farms with more than 250k in subsidies

In [45]:
### copy information to new dataframe gd_dz_viz6
gd_dz_viz6 = gd_dz_groups[gd_dz_groups.count_from250k > 0][["GDE_NAME", "KANTON", "count_from250k", "mean_from250k"]]

In [46]:
### apply remove_kte()
gd_dz_viz6["GDE_CLEAN"] = gd_dz_viz6.GDE_NAME.apply(remove_kte)

In [47]:
### apply clean_nr()
gd_dz_viz6['mean_clean'] = gd_dz_viz6.mean_from250k.apply(clean_nr)

In [74]:
### Look at df
gd_dz_viz6.tail(10)

,GDE_NAME,KANTON,count_from250k,mean_from250k,GDE_CLEAN,mean_clean
1717,Les Verrières,NE,1.0,370465.15,Les Verrières,370'465
1731,Presinge,GE,1.0,268224.70,Presinge,268'225
1737,Bourrignon,JU,1.0,257569.00,Bourrignon,257'569
1741,Courroux,JU,1.0,275073.20,Courroux,275'073
1748,Pleigne,JU,1.0,327272.90,Pleigne,327'273
1751,Haute-Sorne,JU,1.0,308261.85,Haute-Sorne,308'262
1778,Fontenais,JU,1.0,263897.35,Fontenais,263'897
1784,Clos du Doubs,JU,1.0,284685.60,Clos du Doubs,284'686
1785,Haute-Ajoie,JU,1.0,251609.50,Haute-Ajoie,251'610
1786,La Baroche,JU,1.0,274760.65,La Baroche,274'761


In [51]:
### Export data to csv for visualization
gd_dz_viz6.to_csv("data_output/n6_final.csv")

##### average subsidies by municipality

In [65]:
### Create new dataframe
gd_dz_viz5 = gd_dz_groups[["GDE_NAME", "KANTON", "mean", "count_to50k", "count_50to150k", "count_150to250k", "count_from250k"]]


In [69]:
### apply remove_kte()
gd_dz_viz5.loc[:,"GDE_CLEAN"] = gd_dz_viz5.GDE_NAME.apply(remove_kte)

In [68]:
### apply clean_decimal()
gd_dz_viz5.loc[:,'mean_clean'] = gd_dz_viz5["mean"].apply(clean_decimal)

In [71]:
### Look at df
gd_dz_viz5.head(10)

,GDE_NAME,KANTON,mean,count_to50k,count_50to150k,count_150to250k,count_from250k,GDE_CLEAN,mean_clean
0,Aeugst am Albis,ZH,57465.888462,8.0,4.0,1.0,0.0,Aeugst am Albis,57466
1,Affoltern am Albis,ZH,46837.477500,13.0,7.0,0.0,0.0,Affoltern am Albis,46837
2,Bonstetten,ZH,53752.631818,6.0,5.0,0.0,0.0,Bonstetten,53753
3,Hausen am Albis,ZH,64002.022581,13.0,16.0,2.0,0.0,Hausen am Albis,64002
4,Hedingen,ZH,54367.745000,4.0,6.0,0.0,0.0,Hedingen,54368
5,Kappel am Albis,ZH,60305.771053,8.0,11.0,0.0,0.0,Kappel am Albis,60306
6,Knonau,ZH,45068.078571,12.0,9.0,0.0,0.0,Knonau,45068
7,Maschwanden,ZH,66129.656250,8.0,7.0,1.0,0.0,Maschwanden,66130
8,Mettmenstetten,ZH,46601.679730,26.0,11.0,0.0,0.0,Mettmenstetten,46602
9,Obfelden,ZH,47537.312500,11.0,5.0,0.0,0.0,Obfelden,47537


In [73]:
### Export data to csv for visualization
gd_dz_viz5.to_csv("data_output/n5_final.csv")